In [10]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.neighbors import KNeighborsClassifier  #k-近傍法
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール
from sklearn.metrics import mean_squared_error  #機械学習モデルの性能評価のためのモジュール(平均二乗誤差(mean squared error)や決定係数(r2score)など)
import xgboost as xgb #XGBoost
import lightgbm as lgb  #LightGBM
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール
import torch  #PyTorch
import torch.nn as nn #ニューラルネットワークのためのモジュール
import torch.optim as optim #パラメータの最適化を行うためのモジュール
from torch.utils.data import DataLoader, Dataset  #データをバッチ単位でロードするためのユーティリティクラス

from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [11]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #機械学習のトレーニングデータの割合
N_ESTIMATORS = 100  #決定木の数

In [12]:
#ディレクトリ内のAMWS020のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [13]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [14]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [15]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [16]:
#連続する2サンプルの差分を取る関数
def calculate_differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    DifferenceAcc = [math.fabs(ResultantAcc[i + 1] * 100000 - ResultantAcc[i] * 100000) for i in range(len(ResultantAcc) - 1)]  #100000倍して誤差を取る
    return DifferenceAcc

In [17]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [18]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [19]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [20]:
#推定精度を算出する関数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [21]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が全加速度の差分データの最小値〜最大値）
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [22]:
#入力された加速度の差分のリストからヒストグラムを作る関数（ビンの範囲が各加速度の差分データの最小値〜最大値）
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)
    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        #DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS) #ヒストグラムを作成し、同じ数のビンで区切る
    return DifferenceAcc_hist

In [23]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [24]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [25]:
#k-近傍法による機械学習モデル構築と性能評価までを自動化した関数
def k_neighbors(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)
    scaler = StandardScaler()
    scaler.fit(DifferenceAcc_hist)
    newdata = scaler.transform(DifferenceAcc_hist)

    x_train, x_test, y_train, y_test = train_test_split(newdata, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [26]:
#XGBoostによる機械学習モデル構築と性能評価までを自動化した関数
def xgboost(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

    le = LabelEncoder()
    Hz = le.fit_transform(Hz)
    # 学習する
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
    clf = xgb.XGBClassifier(objective='multi:softmax')
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))


In [27]:
path = "my_walk_data/"

In [28]:
#個々実験用
Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)

#使う変数を宣言
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(calculate_differences_of_acceleration(ResultantAcc))

DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)

In [29]:
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, np.array(Hz), train_size = TRAIN_SIZE, shuffle = True)

In [34]:
import tensorflow as tf

# Define the parameters of the model.
d_model = 64  # The dimension of the embedding space.
num_heads = 8  # The number of attention heads.
ff_dim = 128  # The dimension of the feed-forward network.
num_blocks = 4  # The number of transformer blocks.

# Create the embedding layer.
embedding = tf.keras.layers.Embedding(
    input_dim=4000,  # The number of unique tokens in the vocabulary.
    output_dim=d_model,
)

# Create the transformer blocks.
transformer_blocks = []
for _ in range(num_blocks):
    transformer_blocks.append(
        tf.keras.layers.TransformerBlock(
            d_model, num_heads, ff_dim, rate=0.1
        )
    )

# Create the output layer.
output_layer = tf.keras.layers.Dense(1)

# Create the model.
model = tf.keras.models.Sequential([
    embedding,
    *transformer_blocks,
    output_layer
])

# Compile the model.
model.compile(optimizer='adam', loss='mse')

# Train the model.
model.fit(x_train, y_train, epochs=10)

AttributeError: module 'tensorflow.keras.layers' has no attribute 'TransformerBlock'

In [33]:
type(y_train)

numpy.int64

In [ ]:
#尖度
DifferenceAcc_kurtosis = np.zeros(len(DifferenceAcc_hist))
for i in range(len(DifferenceAcc_hist)):
    DifferenceAcc_kurtosis[i] = kurtosis(DifferenceAcc_hist[i])

#歪度
DifferenceAcc_skewness = np.zeros(len(DifferenceAcc_hist))
for i in range(len(DifferenceAcc_hist)):
    DifferenceAcc_skewness[i] = skew(DifferenceAcc_hist[i])

#分散
histogram_var = np.zeros(len(DifferenceAcc_list))
for i in range(len(DifferenceAcc_hist)):
    histogram_var[i] = np.var(DifferenceAcc_list[i])

In [ ]:
#X_newに尖度＋歪度+分散を連結したもの
X_new = np.concatenate((DifferenceAcc_kurtosis.reshape(-1, 1), DifferenceAcc_skewness.reshape(-1, 1), histogram_var.reshape(-1, 1)), axis=1)
#XはDifferenceAcc_hist+X_new
X = np.concatenate((DifferenceAcc_hist, X_new), axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Hz = le.fit_transform(Hz)

In [ ]:
from __future__ import print_function
import torch
x = torch.rand(5, 3)
print(x)

In [ ]:
model = tf.keras.models. Sequential([
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax),
])

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics= ["accuracy"])

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
#model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
# Transformerブロックの実装
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)

    def call(self, inputs):
        x = inputs
        attn_output = self.att(x, x)
        x = x + attn_output
        x = self.layernorm1(x)
        ffn_output = self.ffn(x)
        x = x + ffn_output
        x = self.layernorm2(x)
        x = self.dropout(x)
        return x

# Transformerモデルの構築
class TransformerRegressionModel(tf.keras.Model):
    def __init__(self, d_model, num_heads, ff_dim, num_blocks, input_shape):
        super(TransformerRegressionModel, self).__init__()
        self.input_shape_ = input_shape
        self.embedding = Dense(d_model, input_shape=input_shape)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, ff_dim) for _ in range(num_blocks)]
        self.flatten = tf.keras.layers.Flatten()
        self.fc = Dense(1)

    def call(self, inputs):
        x = self.embedding(inputs)
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [ ]:
# モデルのインスタンス化
d_model = 64  # トランスフォーマーブロックの隠れ層の次元数
num_heads = 4  # Multi-Head Attentionのヘッドの数
ff_dim = 128  # Position-wise Feed-Forward Networkの隠れ層の次元数
num_blocks = 4  # トランスフォーマーブロックの数
input_shape = (100, 3)  # 入力データの形状 (バッチサイズ, シーケンスの長さ, 特徴量の次元数)

model = TransformerRegressionModel(d_model, num_heads, ff_dim, num_blocks, input_shape)

# モデルのコンパイル
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaler.fit_transform(DifferenceAcc_hist), scaler.fit_transform(np.array(Hz)), train_size = TRAIN_SIZE, shuffle = True)

In [ ]:
scaler = StandardScaler()
#X_train = scaler.fit_transform(X)

In [ ]:
type(y_train)

In [ ]:
# モデルの学習
model.fit(x_train, y_train, epochs=10, batch_size=32)

In [ ]:
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, LayerNormalization, MultiHeadAttention, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

def transformer_model(input_shape, d_model=64, num_heads=4, num_layers=4, dropout_rate=0.2):
    """
    Transformer回帰モデルの定義
    """
    inputs = Input(shape=input_shape)

    # Embedding層
    embedding = Embedding(input_dim=input_shape[0], output_dim=d_model)(inputs)
    embedding = Dropout(dropout_rate)(embedding)

    # Positional Encoding
    position = tf.range(start=0, limit=input_shape[1], delta=1)
    position_embedding = Embedding(input_shape[1], d_model)(position)
    embedding = tf.math.add(embedding, position_embedding)

    # Encoder
    encoder = embedding
    for i in range(num_layers):
        encoder = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)([encoder, encoder])
        encoder = Dropout(dropout_rate)(encoder)
        encoder = LayerNormalization(epsilon=1e-6)(encoder)
        ffn = TimeDistributed(Dense(d_model, activation='relu'))(encoder)
        ffn = TimeDistributed(Dense(d_model))(ffn)
        encoder = Dropout(dropout_rate)(ffn)
        encoder = LayerNormalization(epsilon=1e-6)(encoder)

    # Decoder
    decoder = Dense(d_model, activation='relu')(encoder)
    decoder = Dense(1)(decoder)

    model = Model(inputs=inputs, outputs=decoder)
    return model

# ハイパーパラメーターの設定
D_MODEL = 64
NUM_HEADS = 4
NUM_LAYERS = 4
DROPOUT_RATE = 0.2
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 10

# モデルの構築
model = transformer_model(input_shape=(100, 10), d_model=D_MODEL, num_heads=NUM_HEADS, num_layers=NUM_LAYERS, dropout_rate=DROPOUT_RATE)
model.compile(optimizer=Adam(LEARNING_RATE), loss=MeanSquaredError())

# データの読み込みと学習
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
x_train_normalized = normalize(x_train)  # データの正規化
model.fit(x_train_normalized, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Prepare the data
x_train = np.random.rand(100, 10, 3) # 100 sequences of 10 timesteps with 3 features
y_train = np.random.rand(100, 1) # 100 target values

# Define the model architecture
inputs = keras.Input(shape=(10, 3))
attn_output = layers.MultiHeadAttention(num_heads=2, key_dim=2)(inputs, inputs)
attn_output = layers.LayerNormalization(epsilon=1e-6)(attn_output)
ffn_output = layers.Dense(32, activation="relu")(attn_output)
ffn_output = layers.Dense(1)(ffn_output)
outputs = layers.Activation("linear")(ffn_output)
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Make predictions
x_test = np.random.rand(1, 10, 3) # 1 sequence of 10 timesteps with 3 features
y_pred = model.predict(x_test)
print(y_pred)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Prepare the data
x_train = np.linspace(0, 2*np.pi, 1000)
y_train = np.sin(x_train)

# Define the model architecture
inputs = keras.Input(shape=(None, 1))
attn_output = layers.MultiHeadAttention(num_heads=8, key_dim=1)(inputs, inputs)
attn_output = layers.LayerNormalization(epsilon=1e-6)(attn_output)
ffn_output = layers.Dense(32, activation="relu")(attn_output)
ffn_output = layers.Dense(1)(ffn_output)
outputs = layers.Activation("linear")(ffn_output)
model = keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
x_train = x_train.reshape((x_train.shape[0], 1, 1))
y_train = y_train.reshape((y_train.shape[0], 1))
model.fit(x_train, y_train, epochs=100, batch_size=32)

# Make predictions
x_test = np.linspace(0, 2*np.pi, 100)
x_test = x_test.reshape((x_test.shape[0], 1, 1))
y_pred = model.predict(x_test)

# Plot the results
import matplotlib.pyplot as plt
plt.plot(x_train.reshape((-1,)), y_train.reshape((-1,)), label='Training Data')
plt.plot(x_test.reshape((-1,)), y_pred.reshape((-1,)), label='Predictions')
plt.legend()
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Transformerブロックの実装
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)

    def call(self, inputs):
        x = inputs
        attn_output = self.att(x, x)
        x = x + attn_output
        x = self.layernorm1(x)
        ffn_output = self.ffn(x)
        x = x + ffn_output
        x = self.layernorm2(x)
        x = self.dropout(x)
        return x

# Transformerモデルの構築
class TransformerRegressionModel(tf.keras.Model):
    def __init__(self, d_model, num_heads, ff_dim, num_blocks, input_shape):
        super(TransformerRegressionModel, self).__init__()
        self.input_shape_ = input_shape
        self.embedding = Dense(d_model, input_shape=input_shape)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, ff_dim) for _ in range(num_blocks)]
        self.flatten = tf.keras.layers.Flatten()
        self.fc = Dense(1)

    def call(self, inputs):
        x = self.embedding(inputs)
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# モデルのインスタンス化
d_model = 64  # トランスフォーマーブロックの隠れ層の次元数
num_heads = 4  # Multi-Head Attentionのヘッドの数
ff_dim = 128  # Position-wise Feed-Forward Networkの隠れ層の次元数
num_blocks = 4  # トランスフォーマーブロックの数
input_shape = (100, 3)  # 入力データの形状 (バッチサイズ, シーケンスの長さ, 特徴量の次元数)

model = TransformerRegressionModel(d_model, num_heads, ff_dim, num_blocks, input_shape)

# モデルのコンパイル
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)

# モデルの学習
model.fit(x_train, y_train, epochs=10, batch_size=32)


In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# データの作成
x_train = np.random.rand(1000, 10, 3)
y_train = np.random.rand(1000, 1)

# Transformerブロックの実装
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(rate)

    def call(self, inputs):
        x = inputs
        attn_output = self.att(x, x)
        x = x + attn_output
        x = self.layernorm1(x)
        ffn_output = self.ffn(x)
        x = x + ffn_output
        x = self.layernorm2(x)
        x = self.dropout(x)
        return x

# Transformerモデルの構築
class TransformerRegressionModel(tf.keras.Model):
    def __init__(self, d_model, num_heads, ff_dim, num_blocks, input_shape):
        super(TransformerRegressionModel, self).__init__()
        self.input_shape_ = input_shape
        self.embedding = Dense(d_model, input_shape=input_shape)
        self.transformer_blocks = [TransformerBlock(d_model, num_heads, ff_dim) for _ in range(num_blocks)]
        self.flatten = tf.keras.layers.Flatten()
        self.fc = Dense(1)

    def call(self, inputs):
        x = self.embedding(inputs)
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# モデルのインスタンス化
d_model = 64  # トランスフォーマーブロックの隠れ層の次元数
num_heads = 4  # Multi-Head Attentionのヘッドの数
ff_dim = 128  # Position-wise Feed-Forward Networkの隠れ層の次元数
num_blocks = 4  # トランスフォーマーブロックの数
input_shape = (10, 3)  # 入力データの形状 (バッチサイズ, シーケンスの長さ, 特徴量の次元数)

model = TransformerRegressionModel(d_model, num_heads, ff_dim, num_blocks, input_shape)

# モデルのコンパイル
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)

# モデルの学習
model.fit(x_train, y_train, epochs=10)


2023-04-21 15:25:17.764282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
32/32 [==============================] - 8s 25ms/step - loss: 1.3901
Epoch 2/10
32/32 [==============================] - 1s 28ms/step - loss: 0.3821
Epoch 3/10
32/32 [==============================] - 1s 23ms/step - loss: 0.2959
Epoch 4/10
32/32 [==============================] - 1s 22ms/step - loss: 0.2822
Epoch 5/10
32/32 [==============================] - 1s 23ms/step - loss: 0.2659
Epoch 6/10
32/32 [==============================] - 1s 25ms/step - loss: 0.2416
Epoch 7/10
32/32 [==============================] - 1s 21ms/step - loss: 0.2439
Epoch 8/10
32/32 [==============================] - 1s 21ms/step - loss: 0.2086
Epoch 9/10
32/32 [==============================] - 1s 21ms/step - loss: 0.1879
Epoch 10/10
32/32 [==============================] - 1s 23ms/step - loss: 0.2136


In [6]:
import tensorflow as tf
from transformers import AutoModelForSequenceClassification

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import tensorflow as tf
from transformers import AutoModelForSequenceClassification

# Load the transformer model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Create the input features
input_features = tf.io.parse_example(
    train_dataset,
    features={
        "input_text": tf.io.FixedLenFeature([max_seq_len], tf.string),
        "target_label": tf.io.FixedLenFeature([1], tf.float32),
    },
)

# Get the input and target tensors
input_text = input_features["input_text"]
target_label = input_features["target_label"]

# Create the loss function
loss_function = tf.keras.losses.MeanSquaredError()

# Create the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Create the model
model.compile(optimizer=optimizer, loss=loss_function)

# Train the model
model.fit(train_dataset, epochs=10)

# Evaluate the model
model.evaluate(test_dataset)

# Use the model to make predictions
predictions = model.predict(test_dataset)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

NameError: name 'train_dataset' is not defined